In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
import torchvision
import torch.utils.data as td
import matplotlib.pyplot as plt
import torch.nn as nn
from PIL import Image
import torch.nn.functional as F
import gc
import torchvision.datasets as datasets


test_path = '/content/Testing'
train_path = '/content/Training'

preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    ),
])




train_dataset = datasets.GTSRB(
        root='/content', split='train',
        download=True, transform=preprocess,
    )

test_dataset = datasets.GTSRB(
        root='/content', split='test',
        download=True, transform=preprocess,
    )


train_dataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True,  num_workers=2)
print('Train',len(train_dataset))

print('TEST',len(test_dataset))
test_dataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=True,  num_workers=2)

Train 26640
TEST 12630


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 43):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

    num_classes = 43
num_epochs = 20
batch_size = 16
learning_rate = 0.01
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_dataLoader)
training_accuracies = []
training_losses = []
test_accuracies = []
test_losses = []

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataLoader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #accuracy
    total = labels.size(0)
    _,predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    accuracy = (correct / total) * 100

    del images, labels, outputs
    torch.cuda.empty_cache()
    gc.collect()
    accuracy = (correct / total) * 100
    training_accuracies.append(accuracy)
    training_losses.append(loss.item())

    print ('Training set: Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.2f}%, ' 
                .format(epoch+1, num_epochs, loss.item(), accuracy))

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_dataLoader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss_test = criterion(outputs, labels)
            del images, labels, outputs
        accuracy = (correct / total) * 100    
        test_accuracies.append(accuracy)
        test_losses.append(loss_test.item())
        print ('Test set: Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.2f}%, ' 
                    .format(epoch+1, num_epochs, loss_test.item(), accuracy))

Training set: Epoch [1/20], Loss: 2.6172,  Accuracy: 31.25%, 
Test set: Epoch [1/20], Loss: 3.2359,  Accuracy: 15.00%, 
Training set: Epoch [2/20], Loss: 1.5943,  Accuracy: 50.00%, 
Test set: Epoch [2/20], Loss: 2.9287,  Accuracy: 27.79%, 
Training set: Epoch [3/20], Loss: 0.9310,  Accuracy: 68.75%, 
Test set: Epoch [3/20], Loss: 1.4719,  Accuracy: 46.27%, 
Training set: Epoch [4/20], Loss: 0.8341,  Accuracy: 81.25%, 
Test set: Epoch [4/20], Loss: 2.0556,  Accuracy: 58.36%, 
Training set: Epoch [5/20], Loss: 0.6277,  Accuracy: 81.25%, 
Test set: Epoch [5/20], Loss: 3.7545,  Accuracy: 33.15%, 
Training set: Epoch [6/20], Loss: 0.2586,  Accuracy: 93.75%, 
Test set: Epoch [6/20], Loss: 1.7673,  Accuracy: 55.68%, 
Training set: Epoch [7/20], Loss: 0.1196,  Accuracy: 100.00%, 
Test set: Epoch [7/20], Loss: 0.6538,  Accuracy: 81.78%, 
Training set: Epoch [8/20], Loss: 0.0663,  Accuracy: 100.00%, 
Test set: Epoch [8/20], Loss: 0.7022,  Accuracy: 86.33%, 
Training set: Epoch [9/20], Loss: 0.30

In [ ]:
print(f"training losses:{training_losses}")
print(f"training accuracies:{training_accuracies}")
print(f"test losses:{test_losses}")
print(f"test accuracies:{test_accuracies}")

training losses:[2.6171932220458984, 1.5942881107330322, 0.9310141801834106, 0.8341421484947205, 0.6277495622634888, 0.2586292028427124, 0.11962597072124481, 0.06626647710800171, 0.300148069858551, 0.039431288838386536, 0.039000410586595535, 0.03585750237107277, 0.07277382165193558, 0.03557943180203438, 0.05555744469165802, 0.063791923224926, 0.10568579286336899, 0.18903960287570953, 0.1597646027803421, 0.03042464889585972]
training accuracies:[31.25, 50.0, 68.75, 81.25, 81.25, 93.75, 100.0, 100.0, 93.75, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 93.75, 100.0, 93.75, 100.0, 100.0]
test losses:[3.2358694076538086, 2.9286601543426514, 1.471924066543579, 2.0555505752563477, 3.7545244693756104, 1.767329216003418, 0.6538029313087463, 0.7022203207015991, 1.5114637613296509, 0.5701111555099487, 0.3333728015422821, 0.30241963267326355, 1.9915285110473633, 0.43640434741973877, 0.2559519112110138, 0.423153817653656, 0.8821278810501099, 1.18180513381958, 0.5965354442596436, 0.2471122145652771]
te